# Analysis of the Combined data for 7pad Scanning

In [2]:
from single_pad_analysis.analysis_functions import *
from single_pad_analysis.HexDetector import HexDetector

In [3]:
run_number = 325
basedir_path = '../scan_pad_dfs/'
get_pad_csv_path = lambda pool_num, mm_channel: f'{basedir_path}Run{run_number}-Pool{pool_num}-{mm_channel}.csv'

pad_centers_df = pd.read_csv(f'{basedir_path}scan_pad_centers.csv')

In [ ]:
class Pad:
    def __init__(self, pool_num, mm_channel):
        self.pool_num = pool_num
        self.mm_channel = mm_channel
        self.x_center = None
        self.y_center = None

    def __repr__(self):
        return f"Pad(pool_num={self.pool_num}, mm_channel={self.mm_channel}, x_center={self.x_center}, y_center={self.y_center})"

In [ ]:
pads = [
    Pad(4, 'C2'),
]

In [ ]:
# Update pads with pad center information
for pad in pads:
    pad_center_row = pad_centers_df[(pad_centers_df['run_number'] == run_number) & (pad_centers_df['pool_number'] == pad['pool_num']) & (pad_centers_df['channel_number'] == pad['mm_channel'])]
    if pad_center_row.empty:
        print(f"Pad center not found for run {run_number}, pool {pad['pool_num']}, channel {pad['mm_channel']}")
        continue
    elif len(pad_center_row) > 1:
        print(f"Multiple pad centers found for run {run_number}, pool {pad['pool_num']}, channel {pad['mm_channel']}")
        continue
    pad_center_row = pad_center_row[0]
    x_center = Measure(pad_center_row['x_center'], pad_center_row['x_center_err'])
    y_center = Measure(pad_center_row['y_center'], pad_center_row['y_center_err'])
    pad.x_center = x_center
    pad.y_center = y_center

In [ ]:
# Load dataframes for each pad
for pad in pads:
    pad.df = pd.read_csv(get_pad_csv_path(pad.pool_num, pad.mm_channel))
